In [1]:
import opensim as osim
import numpy as np
import clifford as cf
import clifford.tools.g3c as g3c_tools
import common

G3, blades_g3 = cf.Cl(3) # instantiate a 3D- GA
G3c, blades_g3c, stuff = cf.conformalize(G3)

e1, e2, e3 = blades_g3c["e1"], blades_g3c["e2"], blades_g3c["e3"]
up, down = stuff["up"], stuff["down"]
no, ni = stuff["eo"], stuff["einf"]

c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference

In [2]:
model = osim.Model()

ground = model.getGround()
bodyA = osim.Body("bodyA", 1.0, osim.Vec3(0), osim.Inertia(0))
bodyB = osim.Body("bodyB", 2.0, osim.Vec3(0), osim.Inertia(0))
bodyC = osim.Body("bodyC", 2.0, osim.Vec3(0), osim.Inertia(0))

joint0 = osim.GimbalJoint("joint0", ground, osim.Vec3(-1,1,0), osim.Vec3(np.pi/5, np.pi / 3, -np.pi/6), bodyA, osim.Vec3(1,0.5,0.3), osim.Vec3(np.pi/3, -np.pi/3, np.pi/7))
joint1 = osim.PinJoint("joint1", bodyA, osim.Vec3(-1,1,0), osim.Vec3(np.pi/5, np.pi / 3, -np.pi/6), bodyB, osim.Vec3(1,0.5,0.3), osim.Vec3(np.pi/3, -np.pi/3, np.pi/7))
joint2 = osim.GimbalJoint("joint2", bodyB, osim.Vec3(-1,1,0), osim.Vec3(np.pi/5, np.pi / 3, -np.pi/6), bodyC, osim.Vec3(1,0.5,0.3), osim.Vec3(np.pi/3, -np.pi/3, np.pi/7))

muscle = osim.Millard2012EquilibriumMuscle("muscle",  # Muscle name
                                           200.0,  # Max isometric force
                                           0.6,  # Optimal fibre length
                                           0.55,  # Tendon slack length
                                           0.0)  # Pennation angle
origin = osim.Vec3(0.9, 0.8, 0.8)
muscle.addNewPathPoint("origin", ground, origin)

insertion = osim.Vec3(0.7, 0.8, 0.1)
muscle.addNewPathPoint("insertion", bodyC, insertion)

model.addBody(bodyA)
model.addBody(bodyB)
model.addBody(bodyC)
model.addJoint(joint0)
model.addJoint(joint1)
model.addJoint(joint2)
model.addForce(muscle)

state = model.initSystem()

In [3]:
# POSTURE 1
for c in model.getCoordinateSet():
    c.setValue(state, np.random.uniform(-np.pi, np.pi))
model.equilibrateMuscles(state)

lever_arms = np.array([muscle.computeMomentArm(state, c) for c in model.getCoordinateSet()])
print(lever_arms)

J = common.station_jacobian(model, state, osim.Vec3(4,0,3), bodyC)
Jw = common.frame_jacobian(model, state, bodyC)[:3,:]

j = joint1.getParentFrame()
# j = bodyC
O_in_ground = ground.findStationLocationInGround(state, origin).to_numpy()
I_in_ground = bodyC.findStationLocationInGround(state, insertion).to_numpy()

O_in_j1 = ground.findStationLocationInAnotherFrame(state, origin, joint1.getParentFrame()).to_numpy()
I_in_j1 = bodyC.findStationLocationInAnotherFrame(state, insertion, joint1.getParentFrame()).to_numpy()

u_in_ground = I_in_ground - O_in_ground
u1 = I_in_j1 - O_in_j1

print()
print(lever_arms)
ground: osim.Ground = model.getGround()

W1 = ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 3]), joint1.getParentFrame()).to_numpy()
g1 = W1 @ (np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1))
print(g1)
print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 3]), joint1.getParentFrame()).to_numpy().dot(np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1)))

[ 0.87729004  0.20544519 -0.795203    1.0184641   0.21064153 -0.02307136
 -0.27292878]

[ 0.87729004  0.20544519 -0.795203    1.0184641   0.21064153 -0.02307136
 -0.27292878]
1.0184640961793183
1.0184640961793183


In [4]:
# POSTURE k
k = 6
all_states = []
all_lever_arms = []
for _ in range(k):
    state = osim.State(state)
    for c in model.getCoordinateSet():
        c.setValue(state, np.random.uniform(-np.pi, np.pi))
    model.equilibrateMuscles(state)

    lever_arms = muscle.computeMomentArm(state, model.getCoordinateSet().get('joint1_coord_0'))
    all_lever_arms.append(lever_arms)

    J = common.station_jacobian(model, state, osim.Vec3(4,0,3), bodyC)
    Jw = common.frame_jacobian(model, state, bodyC)[:3,:]
    all_states.append(state)

# for s in all_states:
#     print([c.getValue(s) for c in model.getCoordinateSet()])

In [5]:
w = [0, 0, 1]
all_w_in_j1 = []
all_O_in_j1 = []
all_I_in_j1 = []
for i in range(k):
    state = all_states[i]
    Jw = common.frame_jacobian(model, state, bodyC)[:3,:]
    w_ = ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:,3]), joint1.getParentFrame()).to_numpy()
    all_w_in_j1.append(w_)
    
    O_in_j1 = ground.findStationLocationInAnotherFrame(state, origin, joint1.getParentFrame()).to_numpy()
    I_in_j1 = bodyC.findStationLocationInAnotherFrame(state, insertion, joint1.getParentFrame()).to_numpy()
    all_O_in_j1.append(O_in_j1)
    all_I_in_j1.append(I_in_j1)

all_n = []
all_normals = []
all_normals_normalized = []
for i in range(k):
    state = all_states[i]
    w_ = all_w_in_j1[i]

    O_in_j1 = all_O_in_j1[i]
    I_in_j1 = all_I_in_j1[i]

    n = w_.dot(np.cross(I_in_j1, O_in_j1) / np.linalg.norm(I_in_j1 - O_in_j1))
    normals = np.cross(I_in_j1, O_in_j1)
    normals_normalized = np.cross(I_in_j1, O_in_j1) / np.linalg.norm(I_in_j1 - O_in_j1)
    all_normals.append(normals)
    all_normals_normalized.append(normals_normalized)
    # print(n)
    all_n.append(n)
print(all_lever_arms)
print(all_n)

[0.6200146326784648, 1.2692587827731925, -0.796188753050032, -0.9082241139758042, -0.5321503725730591, 0.41193317725600925]
[0.6200146326784648, 1.269258782773193, -0.7961887530500321, -0.9082241139758046, -0.532150372573059, 0.41193317725600914]


In [6]:
Base = np.array([e1, e2, e3])
O = origin.to_numpy() @ Base
I = insertion.to_numpy() @ Base
w = e3 # in j1!

alphas = []
n_by_alpha = []
for i in range(k):
    n = all_normals_normalized[i]
    dual_plane = n @ Base
    plane = dual_plane.dual()

    # print(((w^ni) | dual_plane))
    # print(((w^ni) | dual_plane))

    n1 = all_normals_normalized[0]
    alpha = all_lever_arms[0] / all_lever_arms[i]
    alphas.append(alpha)
    ni_adapted = alpha * n
    n_by_alpha.append(alpha * all_normals[i])
    # print(all_lever_arms[0], alpha*all_lever_arms[i])
    
    comb = n1 - ni_adapted
    # print(comb)
    if i != 0:
        # print((comb / np.linalg.norm(comb)).round(2))
        c = comb / np.linalg.norm(comb)
        print(c)
    #    print(np.cross(comb, [0,0,1]))
    #    print(n1/np.linalg.norm(n1))

        pl = (all_normals_normalized[0]@Base) ^ (c@Base) ^ ni
        # print((all_normals_normalized[i]@Base) ^ pl) # All ni belong to the same plane (but they are all different)
        # confirmed by np.linalg.matrix_rank(np.array(all_normals).T) = 2

        # print(e3 ^ pl)
        # print( (all_normals_normalized[i-1]@Base) ^ n)

[-7.85129038e-02  9.96913097e-01 -1.84710187e-16]
[-4.62984750e-01  8.86366245e-01  9.57801436e-16]
[-0.11028599  0.99389989  0.        ]
[-4.68822154e-01  8.83292584e-01  1.95434497e-16]
[-6.57895680e-01  7.53109072e-01  1.43386699e-16]


In [7]:
all_lambdas = []
for i in range(6):
    n1 = all_normals_normalized[0]
    if i > 0:
        n = all_normals_normalized[i]
        a = alphas[i]
        v = n1 - a*n
        print(v / np.linalg.norm(v))
        lambda_i = np.linalg.norm(v)
        all_lambdas.append(lambda_i)
        # print(lambda_i)

        # so suppose we have n = an/a.
        
    # print(n*a)
    # print(n_by_alpha[i])

[-7.85129038e-02  9.96913097e-01 -1.84710187e-16]
[-4.62984750e-01  8.86366245e-01  9.57801436e-16]
[-0.11028599  0.99389989  0.        ]
[-4.68822154e-01  8.83292584e-01  1.95434497e-16]
[-6.57895680e-01  7.53109072e-01  1.43386699e-16]


In [8]:
an = np.array([alphas[i] * all_normals_normalized[i] for i in range(6)])
an
print(an)
barycenter = an.mean(axis=1)
barycenter

an_adapted = an / barycenter.reshape((-1,1))
an_adapted


[[-0.03279135  1.159331    0.62001463]
 [ 0.01439978  0.56012427  0.62001463]
 [ 0.07454119  0.95384703  0.62001463]
 [ 0.15085484 -0.49569273  0.62001463]
 [ 0.7661932  -0.34600983  0.62001463]
 [ 2.0048061  -1.17315601  0.62001463]]


array([[-0.05632465,  1.9913455 ,  1.06497915],
       [ 0.03616404,  1.40671277,  1.5571232 ],
       [ 0.13566075,  1.73594767,  1.12839158],
       [ 1.64463217, -5.4040839 ,  6.75945173],
       [ 2.20975199, -0.99791529,  1.7881633 ],
       [ 4.14311804, -2.42443587,  1.28131783]])

In [9]:
%matplotlib qt5
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

n1 = all_normals_normalized[0]
for i in range(6):
    # print(n)
    # ax.plot([0, n[0]], [0, n[1]], [0, n[2]])

    if i > 0:
        n = all_normals_normalized[i]
        # n = an_adapted[i]
        a = alphas[i]
        v = n1 - a*n
        v = a*n
        # v = n
        # v = n1 - n
        
        # v = v / np.linalg.norm(v)
        # print(v / np.linalg.norm(v))
        # lambda_i = np.linalg.norm(v)
        # all_lambdas.append(lambda_i)
        ax.plot([0, v[0]], [0, v[1]], [0, v[2]])
        ax.scatter(*v)
        # print(lambda_i)

ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.set_zlim(-2,2)
# n = n_by_alpha[1]
# ax.plot([0, n[0]], [0, n[1]], [0, n[2]], c="red")

(-2.0, 2.0)

In [10]:
print(n1 @ np.array([0,0,1]))
print(all_lever_arms[0])
# g1
print(alphas[2]*all_normals_normalized[2] @ np.array([0,0,1]))

0.6200146326784649
0.6200146326784648
0.6200146326784647


In [11]:
print(1/np.linalg.norm(all_normals_normalized[0]))

0.7603866253176814


In [12]:
# Let's displace normal planes to the origin and insertion frame

all_planes_in_origin = []
all_planes_in_insertion = []
for i, n in enumerate(all_normals_normalized):
    p0, p1 = [0,0,0], n.tolist()
    state = all_states[i]
    # print(ni)
    
    
    p0_in_origin = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(0), ground).to_numpy()
    p1_in_origin = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*p1), ground).to_numpy()
    
    p0_in_insertion = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(0), bodyC).to_numpy()
    p1_in_insertion = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*p1), bodyC).to_numpy()
    
    p0_in_origin = p0_in_origin@Base
    p1_in_origin = p1_in_origin@Base

    p0_in_insertion = p0_in_insertion@Base
    p1_in_insertion = p1_in_insertion@Base

    normal_origin = p1_in_origin - p0_in_origin
    normal_insertion = p1_in_insertion - p0_in_insertion
    # print(p1_in_origin | normal_origin)
    plane_in_origin = (normal_origin + (p0_in_origin | normal_origin)*ni).dual()
    # plane_in_origin = (normal_origin).dual()

    plane_in_insertion = (normal_insertion + (p0_in_insertion | normal_insertion)*ni).dual()

    all_planes_in_origin.append(plane_in_origin)
    all_planes_in_insertion.append(plane_in_insertion)
    # ni_ = ni @ Base
    # print(ni_)

c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [626]:
print(up(origin.to_numpy()@Base) ^ all_planes_in_origin[0])
print(up(origin.to_numpy()@Base) ^ all_planes_in_origin[1])
print(up(origin.to_numpy()@Base) ^ all_planes_in_origin[2])
print(up(origin.to_numpy()@Base) ^ all_planes_in_origin[3])

print(up(insertion.to_numpy()@Base) ^ all_planes_in_insertion[0])
print(up(insertion.to_numpy()@Base) ^ all_planes_in_insertion[1])
print(up(insertion.to_numpy()@Base) ^ all_planes_in_insertion[2])
print(up(insertion.to_numpy()@Base) ^ all_planes_in_insertion[3])
# (origin.to_numpy()@Base) ^ all_inter_origin[1]

0
0
0
0
0
0
0
0


In [639]:
for i, n in enumerate(all_normals_normalized):
    if i > 0:
        n = all_normals_normalized[i]
        # n = an_adapted[i]
        a = alphas[i]
        v = n1 - a*n
        v = a*n
        c = v - n1
        # print(c)
        print(np.cross(c/np.linalg.norm(c), n), n)

[0.45759499 0.44214872 1.24245095] [ 0.26926895 -2.06672172  0.63630863]
[0.04705381 0.01550573 0.08504874] [ 0.57461876 -1.47200083 -0.0495428 ]
[0.45226263 0.19319896 0.88041509] [-0.51609665 -1.03301276  0.49180009]
[ 0.22544231 -0.6858291  -0.67614428] [-2.19611259 -0.0101578  -0.72193198]
[ 1.47775618 -0.67421512  1.01562993] [ 1.53609754  0.92002762 -1.62429349]


In [671]:
w = e3
g1 = all_lever_arms[0]
# Pw_at_g1 = up(g1 * w)^e1^e2^ni
Pw_at_g1 = (w + (g1*w | w)*ni).dual()
print(up(g1*w) ^ Pw_at_g1)

# random_line = 

0
